In [28]:
import tensorflow as tf
import os
import zipfile
import pandas as pd
import numpy as np
tf.__version__

'2.13.0'

In [29]:
!kaggle datasets download -d datatattle/covid-19-nlp-text-classification -p Datasets_Collection/Coronavirus_Tweets_Classification


  0%|          | 0.00/4.38M [00:00<?, ?B/s]
 23%|██▎       | 1.00M/4.38M [00:01<00:05, 644kB/s]
 46%|████▌     | 2.00M/4.38M [00:01<00:02, 1.18MB/s]
 68%|██████▊   | 3.00M/4.38M [00:02<00:00, 1.66MB/s]
 91%|█████████▏| 4.00M/4.38M [00:02<00:00, 2.12MB/s]
100%|██████████| 4.38M/4.38M [00:02<00:00, 1.72MB/s]


In [30]:
dataset_dir = os.path.join(os.getcwd(), "Datasets_Collection", "Coronavirus_Tweets_Classification")

In [31]:
zip_found = False
for filename in os.listdir(dataset_dir):
    if filename.endswith(".zip"):
        dataset_zip_name = filename
        zip_found = True
        break

if not zip_found:
    raise FileNotFoundError()

In [32]:
with zipfile.ZipFile(os.path.join(dataset_dir, dataset_zip_name), 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

In [34]:
train_df = pd.read_csv(os.path.join(dataset_dir, "Corona_NLP_train.csv"), encoding = "ISO-8859-1")
test_df = pd.read_csv(os.path.join(dataset_dir, "Corona_NLP_test.csv"), encoding = "ISO-8859-1")

In [35]:
print(train_df)

       UserName  ScreenName                      Location     TweetAt  \
0          3799       48751                        London  16-03-2020   
1          3800       48752                            UK  16-03-2020   
2          3801       48753                     Vagabonds  16-03-2020   
3          3802       48754                           NaN  16-03-2020   
4          3803       48755                           NaN  16-03-2020   
...         ...         ...                           ...         ...   
41152     44951       89903  Wellington City, New Zealand  14-04-2020   
41153     44952       89904                           NaN  14-04-2020   
41154     44953       89905                           NaN  14-04-2020   
41155     44954       89906                           NaN  14-04-2020   
41156     44955       89907  i love you so much || he/him  14-04-2020   

                                           OriginalTweet           Sentiment  
0      @MeNyrbie @Phil_Gahan @Chrisitv https